In [1]:
import os
import warnings

warnings.filterwarnings("ignore")



files = []

for filename in os.listdir("data"):
    
    with open("data/" + filename, encoding="UTF-8") as f:
        files.append(f.read())

In [2]:
validation_text = " ".join(files[-3:])
train_text = " ".join(files[:-3])

f = open("train_dataset.txt", 'w', encoding="UTF-8")
f.write(train_text)
f.close()

f = open("test_dataset.txt", 'w', encoding="UTF-8")
f.write(validation_text)
f.close()

In [3]:
#Could be changed to other models
model_name = "gpt2-medium"

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [6]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained(model_name)


training_args = TrainingArguments(
    output_dir="./gpt2-trump", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [7]:
trainer.train()

Step,Training Loss
500,2.796303


TrainOutput(global_step=633, training_loss=2.7388785363750245)

In [8]:
trainer.save_model("model/")